In [39]:

#import nessecary package
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
%pip install geopy
%pip install folium
%pip install geocoder
import geocoder
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [168]:
#url for data extarct
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [169]:

#get the data from url
page = requests.get(link)

In [170]:
#extract table from html
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [172]:
#extract columns list
col = []
for i in range(len(tr_elements[0])):
    col.append(tr_elements[0][i].text_content().strip('\n'))
col[0]='Postcode'
col[2]='Neighbourhood'
col

['Postcode', 'Borough', 'Neighbourhood']

In [173]:
#extract table data
data = []
for i in range(1,len(tr_elements)):
    l = []
    if len(tr_elements[i])!=3:
        break
    for o in range(len(tr_elements[i])):
        l.append(tr_elements[i][o].text_content().strip('\n'))
    data.append(l)

In [174]:
#create dataframe
df = pd.DataFrame(data,columns=col)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [175]:
#remove row that borough are not assigned
df = df[df.Borough != 'Not assigned']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [176]:
#join row that have the same postcode
df = df.groupby(['Postcode','Borough'], sort=False).agg( ','.join)
df.reset_index(inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [177]:
df[df.iloc[:,2] == 'Not assigned'] #check if there are any not assigned values

,Postcode,Borough,Neighbourhood


In [178]:
#assign name to Neighbourhood that are not assigned
for i in range(1,len(df['Neighbourhood'])):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]

In [179]:
df[df.iloc[:,2] == 'Not assigned'] #check again if there are any not assigned values

,Postcode,Borough,Neighbourhood


In [180]:
#shape of data
df.shape

(104, 3)

## Second Part

In [181]:

#load latitude and longitude data
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [182]:
#merge latitude and longitude data to original dataframe
df_merged = df.merge(geo_data,how='outer',left_on='Postcode', right_on='Postal Code')

In [183]:
#drop duplicate postal code column
df_merged = df_merged.drop('Postal Code',axis=1)
df_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [185]:
df_merged=df_merged.dropna()
df_merged.shape

(103, 5)

## Third Part

In [186]:

#find latitude and longitude of toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [187]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [188]:
#slice only data from North York
north_york = df_merged[df_merged['Borough'] == 'North York'].reset_index(drop=True)
north_york.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [189]:
#find latitude and longitude of North York
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [190]:
# create map of North York using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york['Latitude'], north_york['Longitude'], north_york['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

In [192]:
#Define argument that need for collect data from foursquare api
CLIENT_ID = 'FCUZKUDRBSQ21D2E1FNXQUAFYCSKFQTOJJ2VGGYXCQQBZI0J' # your Foursquare ID
CLIENT_SECRET = 'EWNJMXKI33YNYPHFN04ZJQIYCMZ2V4LFT2VFYJQX1NKADAW4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FCUZKUDRBSQ21D2E1FNXQUAFYCSKFQTOJJ2VGGYXCQQBZI0J
CLIENT_SECRET:EWNJMXKI33YNYPHFN04ZJQIYCMZ2V4LFT2VFYJQX1NKADAW4


In [193]:
#Define function for get venue list and categories
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for x in venues_list for item in x])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [194]:
#Use function that we define earlier
north_york_venues = getNearbyVenues(names=north_york['Neighbourhood'],
                                   latitudes=north_york['Latitude'],
                                   longitudes=north_york['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [195]:
#shape of north york venues dataframe
print(north_york_venues.shape)

(238, 7)


In [196]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 100 uniques categories.


In [197]:

# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighbourhood'] = north_york_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + north_york_onehot.columns[:-1].tolist()
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [198]:
#shape of north york after one hot encoding
north_york_onehot.shape

(238, 101)

In [199]:
#groupby neighbourhood and use mean function
north_york_grouped = north_york_onehot.groupby('Neighbourhood').mean().reset_index()
north_york_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.00,0.05,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.05,0.000000,0.05,0.000000,0.0,0.000000,0.050000,0.00,0.000000,0.000000,0.0,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.050000,0.000000,0.050000,0.000000,0.000000,0.000000,0.050000,0.0,0.050000,0.000000,0.0,0.0,0.000000,0.000000,0.050000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.041667,0.041667,0.00,0.00,0.000000,0.083333,0.041667,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.041667,0.041667,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.041667,0.000000,0.083333,0.000000,0.000000,0.041667,0.041667,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.0,0.041667,0.000000,0.0,0.0,0.041667,0.000000,0.083333,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.041667,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.04,0.000000,0.0

In [200]:

north_york_grouped.shape

(20, 101)

In [201]:

#show top venue categories
num_top_venues = 10

for hood in north_york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2                Pizza Place  0.05
3                Bridal Shop  0.05
4              Grocery Store  0.05
5                Gas Station  0.05
6        Fried Chicken Joint  0.05
7  Middle Eastern Restaurant  0.05
8          Mobile Phone Shop  0.05
9                      Diner  0.05


----Bayview Village----
                      venue  freq
0        Chinese Restaurant  0.25
1       Japanese Restaurant  0.25
2                      Bank  0.25
3                      Café  0.25
4         Accessories Store  0.00
5  Mediterranean Restaurant  0.00
6                  Pharmacy  0.00
7                 Pet Store  0.00
8                      Park  0.00
9             Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.08
1               Restaurant  0.08
2       Italian 

In [202]:

#define function to return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [203]:


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = north_york_grouped['Neighbourhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)


neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Sandwich Place,Ice Cream Shop,Diner,Deli / Bodega,Middle Eastern Restaurant,Mobile Phone Shop,Fried Chicken Joint
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Electronics Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Restaurant,Grocery Store,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,Comfort Food Restaurant
3,Don Mills,Beer Store,Coffee Shop,Asian Restaurant,Gym,Japanese Restaurant,Restaurant,Chinese Restaurant,Dim Sum Restaurant,Clothing Store,Caribbean Restaurant
4,Downsview,Grocery Store,Park,Food Truck,Liquor Store,Electronics Store,Snack Place,Baseball Field,Bank,Shopping Mall,Athletics & Sports
5,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Bakery,Bank,Convenience Store,Food Court,Japanese Restaurant
6,Glencairn,Metro Station,Asian Restaurant,Pub,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
7,Hillcrest Village,Golf Course,Dog Run,Pool,Athletics & Sports,Mediterranean Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
8,Humber Summit,Pizza Place,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner
9,"Humberlea, Emery",Baseball Field,Women's Store,Event Space,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store


In [204]:

# set number of clusters
kclusters = 5

north_york_grouped_clustering = north_york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 0, 4], dtype=int32)

In [205]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [206]:
#results dataframe
north_york_merged.dropna(axis=0,inplace=True)
north_york_merged.reset_index(inplace=True)
north_york_merged

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Pizza Place,Hockey Arena,French Restaurant,Portuguese Restaurant,Coffee Shop,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Clothing Store,Sporting Goods Shop,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Accessories Store,Vietnamese Restaurant,Arts & Crafts Store
3,3,M3B,North York,Don Mills,43.745906,-79.352188,2,Beer Store,Coffee Shop,Asian Restaurant,Gym,Japanese Restaurant,Restaurant,Chinese Restaurant,Dim Sum Restaurant,Clothing Store,Caribbean Restaurant
4,4,M6B,North York,Glencairn,43.709577,-79.445073,2,Metro Station,Asian Restaurant,Pub,Japanese Restaurant,Women's Store,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
5,5,M3C,North York,Don Mills,43.725900,-79.340923,2,Beer Store,Coffee Shop,Asian Restaurant,Gym,Japanese Restaurant,Restaurant,Chinese Restaurant,Dim Sum Restaurant,Clothing Store,Caribbean Restaurant
6,6,M2H,North York,Hillcrest Village,43.803762,-79.363452,2,Golf Course,Dog Run,Pool,Athletics & Sports,Mediterranean Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
7,7,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,2,Coffee Shop,Bank,Grocery Store,Sandwich Place,Ice Cream Shop,Diner,Deli / Bodega,Middle Eastern Restaurant,Mobile Phone Shop,Fried Chicken Joint
8,8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,2,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Bakery,Bank,Convenience Store,Food Court,Japanese Restaurant
9,9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,2,Coffee Shop,Furniture / Home Store,Caribbean Restaurant,Bar,Massage Studio,Dog Run,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [207]:

#change type of cluster columns from float to int
north_york_merged['Cluster Labels'] = north_york_merged['Cluster Labels'].astype(int)

In [208]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighbourhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [209]:
#First cluster
north_york_merged.loc[north_york_merged['Cluster Labels'] == 0,north_york_merged.columns[[3,7,8,9,10,11]]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,Humber Summit,Pizza Place,Women's Store,Dog Run,Construction & Landscaping,Convenience Store


In [210]:
#Second cluster
north_york_merged.loc[north_york_merged['Cluster Labels'] == 1,north_york_merged.columns[[3,7,8,9,10,11]]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Parkwoods,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Dog Run
12,"York Mills, Silver Hills",Park,Women's Store,Dog Run,Construction & Landscaping,Convenience Store
14,"North Park, Maple Leaf Park, Upwood Park",Park,Construction & Landscaping,Bakery,Electronics Store,Convenience Store
22,York Mills West,Park,Construction & Landscaping,Convenience Store,Women's Store,Dog Run


In [211]:
#Third cluster
north_york_merged.loc[north_york_merged['Cluster Labels'] == 2,north_york_merged.columns[[3,7,8,9,10,11]]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Victoria Village,Pizza Place,Hockey Arena,French Restaurant,Portuguese Restaurant,Coffee Shop
2,"Lawrence Manor, Lawrence Heights",Clothing Store,Sporting Goods Shop,Furniture / Home Store,Event Space,Miscellaneous Shop
3,Don Mills,Beer Store,Coffee Shop,Asian Restaurant,Gym,Japanese Restaurant
4,Glencairn,Metro Station,Asian Restaurant,Pub,Japanese Restaurant,Women's Store
5,Don Mills,Beer Store,Coffee Shop,Asian Restaurant,Gym,Japanese Restaurant
6,Hillcrest Village,Golf Course,Dog Run,Pool,Athletics & Sports,Mediterranean Restaurant
7,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Sandwich Place,Ice Cream Shop
8,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store
9,"Northwood Park, York University",Coffee Shop,Furniture / Home Store,Caribbean Restaurant,Bar,Massage Studio
10,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store


In [212]:
#Fourth cluster
north_york_merged.loc[north_york_merged['Cluster Labels'] == 3,north_york_merged.columns[[3,7,8,9,10,11]]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
16,"Willowdale, Newtonbrook",Piano Bar,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


In [213]:

#Five cluster
north_york_merged.loc[north_york_merged['Cluster Labels'] == 4,north_york_merged.columns[[3,7,8,9,10,11]]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,"Humberlea, Emery",Baseball Field,Women's Store,Event Space,Convenience Store,Cosmetics Shop
